##### Copyright 2022 Huawei TECHNOLOGIES CO.LTD

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Black-Scholes-Merton期权公式: 定价、隐波动率和希腊值求解

In [2]:
# !pip install --upgrade mindspore
# !pip install mind-quant-finance

In [3]:
# @title temporary install
!pip install -e. --upgrade

Obtaining file:///home/michael/Documents/MindFinance/mind-quant/mind-quant-finance-brownian
  Attempting uninstall: MindQuantFinance
    Found existing installation: MindQuantFinance 0.0.1
    Uninstalling MindQuantFinance-0.0.1:
      Successfully uninstalled MindQuantFinance-0.0.1
  Running setup.py develop for MindQuantFinance


In [4]:
#@title Imports

import matplotlib.pyplot as plt
import numpy as onp

import mindspore as ms
import mindspore.numpy as np
from mindspore import dtype as mstype

from mind_quant_finance.engine.analytic.implied_vol import implied_vol_solver
from mind_quant_finance.engine.analytic.european_option import AnalyticBlackScholesMerton
from mind_quant_finance.math.grad import FirstOrderGrad, SecondOrderGrad

## 基于Black-Scholes-Merton公式的期权定价

### BSM 定价公式

布莱克-斯托尔斯-莫顿公式的基本思路是构造一个期权与标的所组成的无风险交易组合，即该收益组合不存在不确定性。在市场无套利的前提下，该无风险组合的收益应当等于无风险利率带来的收益。利用该等式关系，结合伊藤微积分公式等数学工具，可以解出期权价格的解析解。

设$S_0$为标的现值价格(spots)，$T$为合约时长(expires，单位为年)，$K$为执行价格(strikes)，$\sigma^2$为标的价格波动率(volatilities)。

则BSM公式假设表的价格$S$服从：

$$dS = \mu S dt + \sigma S dz$$

其中$\mu$为标的期望收益率，$dz$服从纳维过程。设期权价格为$f$，无风险利率为$r$，通过构造无风险组合，可以获得BSM微分方程：

$$\frac{\partial f}{\partial t} + rS\frac{\partial f}{\partial S} + \frac{1}{2} \frac{\partial^2 f}{\partial S^2} \sigma^2 S^2 = rf$$

其中，当期权为看涨期权时，$f(t)=max(S(t)-K,0)$；当期权为看跌期权时，$f(t)=max(K-S(t),0)$

解上述BSM微分方程，即可得到最终的BSM定价公式：

$$Call_{price} = S_0 N(d_1) - K e^{-rT}N(d_2)$$
$$Put_{price} = Ke^{-rT}N(-d_2) - S_0 N(-d_1) $$

其中，N(x)为标准正态分布的累计概率分布函数(cdf)。上式中的$d_1$与$d_2$由下式决定：

$$d_1 = \frac{ln(S_0/K)+(r+\sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = \frac{ln(S_0/K)+(r-\sigma^2/2)T}{\sigma\sqrt(T)}=d_1 - \sigma\sqrt{T}$$

### 使用Mind-Quant-Finance进行BSM期权定价

使用Mind-Quant-Finance能够进行大规模并行的BSM期权定价。具体而言，用户每次输入任意大小的Tensor，求解器将输出同样大小的Tensor，该Tensor的每一位表示对应位置的输入值进行BSM定价的结果。要使用Mind-Quant-Finance的高性能BSM定价功能。首先需要定义BlackScholesMerton求解器。BlackScholesMerton求解器包含三个可选参数：

 - is_call_options：与输入值大小相同的Tensor，或者bool变量，其中True表示看涨期权，False表示看跌期权。默认为True
 - is_normal_volatility：是否存在负期权价格。当存在负期权价格时，求解器将使用Bachlier定价模型求解。默认为False
 - dtype：求解数据精度，默认为mstype.float32
 
 
 例如，一个输入Tensor大小为(10,)，其中前五个为看涨期权，后五个为看跌齐全，且不存在负波动率，求解精度为mstype.float32的求解器为：

In [5]:
is_call_options = np.array([True] * 5 + [False] * 5)
bsm_solver = AnalyticBlackScholesMerton(is_call_options=is_call_options)

在定义求解器后，即可使用该求解器进行BSM定价。求解的输入参数包括：

 - expiries：合约时长。
 - strikes：执行价格。
 - spots：标的现值价格。
 - volatilities：标的价格波动率。
 - discounted_rates ：Optional，可选项。复利无风险利率，默认为0。
 - dividend_rates：Optional，可选项。复利股息率，默认为0。


一个求解案例如下：

In [6]:
spots = np.array([80.0, 90.0, 100.0, 110.0, 120.0] * 2, dtype=mstype.float32)  # 输入标的现值(共五种)
strikes = np.array([100.0] * 10, dtype=mstype.float32)  # 执行价格为100
discounted_rates = ms.Tensor(0.08, dtype=mstype.float32)  # 无风险利率/折现率为8%
volatilities = ms.Tensor(0.2, dtype=mstype.float32)     # 标的价格波动率为20%
expiries = ms.Tensor(0.25, dtype=mstype.float32)        # 合约时长为3个月

computed_prices = bsm_solver(
    expiries=expiries,
    strikes=strikes,
    spots=spots,
    volatilities=volatilities,
    discounted_rates=discounted_rates)

print(f"bsm_price_call={computed_prices[:5]}")  # 看涨期权价格
print(f"bsm_price_put={computed_prices[5:]}")  # 看跌期权价格

bsm_price_call=[ 0.06901627  1.0254538   5.016982   12.620461   22.066563  ]
bsm_price_put=[18.088873    9.045315    3.036842    0.6403164   0.08641949]


### 基于BSM公式求解隐波动率

在布莱克-斯托尔斯-莫顿公式中，无法直接观测到的值只有标的价格波动率(volatilities)。在实际中，交易员通常使⽤所谓的隐含波动率(implied
volatility)，这⼀波动率是指由期权的市场价格所隐含的波动率。隐含波动率可以用来衡量市场对于某一标的波动率的观点，不同于历史波动率估计，隐含波动率为一种前瞻型估计。

在Mind-Quant-Finance中，使用牛顿迭代法求解隐含波动率，具体而言，设$f(\sigma)$为波动率-期权价格函数，则迭代公式如下：

$$\sigma_{t+1} = \sigma_t - \frac{f(\sigma_t)}{f'(\sigma_t)}$$

使用Mind-Quant-Finance能够进行大规模并行的隐含波动率计算。具体来说，用户可以调用implied_vol_solver函数进行计算。

implied_vol_solver函数包含以下输入参数：

 - expiries：合约市场，以年为单位。
 - strikes：执行价格。
 - der_prices：期权的现值价格。
 - spots：标的的现值价格。
 - discounted_factors & discounted_rates：可选项，用于衡量折现率，二者最多输入一个。其中discounted_factors为折现因子，discounted_rates为复利无风险利率，二者的转化公式为：discounted_factors = exp(-discounted_rates * expiries)。若二者都未被输入，则默认折现率为1/无风险利率为0。
 - is_call_options：可选项，用于标记Tensor的某一位的期权类型，其中True为看涨期权，False为看跌期权，默认为True。
 - initial_volatilities：可选项，牛顿法的求解起点。默认为$f\sqrt{2\pi}$，其中$f$即期权现值(der_prices)。
 - underlying_distribution：可选项，求解隐藏波动率基于的定价模型，包含BSM和Bachelier两种，详细说明见下文。
 - tolerance：可选项，牛顿法的求解精度，默认为1e-6。
 - max_iterations：可选项，牛顿法的最大迭代次数，默认为20。
 - dtype：求解数据精度，默认为mstype.float32
 
在上述参数中，underlying_distribution参数用于标识求解使用的模型。当期权价格一定为正数时，应当使用BSM定价公式进行求解。当期权价格可能出现负数时，应当使用Bachelier定价公式进行求解。通过加载ImpliedVolUnderlyingDistribution枚举数组(Enum数组)，用户可以方便的选择模型：

In [7]:
# from mind_quant_finance.engine.analytic.implied_vol import ImpliedVolUnderlyingDistribution
# help(ImpliedVolUnderlyingDistribution)

implied_vol_solver函数在求解答案后，会返回一个三元组(implied_vols, converged, failed)，其中：

 - implied_vols：求解结果，即隐波动率。
 - converged：用于标识每个位置的结果是否已经收敛。
 - failed：用于标记某个位置是否求解失败，即发散到inf。
 
基于上文中AnalyticBlackScholesMerton求解器进行BSM定价的结果，一个隐含波动率求解实例如下：

In [9]:
init_vols = np.ones(10) * 0.5
implied_vols, converged, failed = \
    implied_vol_solver(expiries=expiries,
                       strikes=strikes,
                       der_prices=computed_prices,
                       spots=spots,
                       discounted_rates=discounted_rates,
                       is_call_options=is_call_options,
                       initial_volatilities=init_vols,
                       )

print(f"implied_vols = {implied_vols}")
print(f"converged = {converged}")
print(f"failed = {failed}")

implied_vols = [0.1999999  0.20000082 0.20000006 0.20000109 0.20000155 0.19999297
 0.19999951 0.19999947 0.19999927 0.19999646]
converged = [ True  True  True  True  True False  True  True  True  True]
failed = [False False False False False False False False False False]


### 希腊值简介

⾦融机构在场外市场向客户卖出期权后会⾯临风险管理的问题，而期权的风险管理主要指对冲该期权的风险敞口。希腊值(Greek Letters，Greek)是解决该问题的重要工具，每一个希腊值用于度量期权头寸的某种特定风险。对于交易员而言，在多数情况下，管理期权的风险敞口可以转化为对期权希腊值的管理。

常用的希腊值主要包括：Delta、Gamma、Theta、Vega、Rho和Psi。设期权定价公式为：$f(t,S,K,\sigma^2,r,q)$，其中$t$为合约时长(expiries)，$S$为标的现值(spots)，$K$为执行价格(strikes)，$\sigma^2$为标的价格波动率(volatilities)，$r$为无风险利率(discounted_rates)，$q$为股息率(dividend_rates)，则希腊值的定义为：

\begin{aligned}
&Delta = \frac{\partial f}{\partial S} \\
&Gamma = \frac{\partial^2 f}{\partial S^2} \\
&Vega = \frac{\partial f}{\partial (\sigma^2)} \\
&Theta = \frac{\partial f}{\partial t} \\
&Rho = \frac{\partial f}{\partial r} \\
&Psi = \frac{\partial f}{\partial q} 
\end{aligned}

一般而言，通过构建资产组合，使得各希腊值接近零，即可实现风险中性。上述希腊值中，最为重要的为$Delta$，其衡量期权价值相对于标的价格的变化程度。当整个组合的$Delta$之和为0时，称该组合为$Delta$中性。希腊值会随着市场变化而不断变化，交易员通过定期修正当前资产组合，使得希腊值保持接近于零的状态，从而控制风险敞口。

### 使用Mind-Quant-Finance求解希腊值

由于希腊值本质上是期权定价函数对某输入的导数，故在定义了期权定价求解器后，使用mind_quant_finance.math提供的梯度求解工具FirstOrderGrad和SecondOrderGrad，我们可以轻松求得上述所有希腊值。FirstOrderGrad(求一阶梯度)和SecondOrderGrad(求二阶梯度)具有完全相同的参数/输入形式。下面以基于BSM定价公式求解Delta为例。首先，需要定义一阶梯度求解器FirstOrderGrad：

In [10]:
computed_prices = bsm_solver(
    expiries=expiries,
    strikes=strikes,
    spots=spots,
    volatilities=volatilities,
    discounted_rates=discounted_rates)

grad1_solver = FirstOrderGrad(model=bsm_solver, argnums=2)

其中，FirstOrderGrad和SecondOrderGrad有两个参数：

 - model：需要对其进行求导的模型(需要继承自mindspore.nn.Cell)，这里即期权定价求解器
 - argnums：int类型，表示对模型的哪一位输入求梯度
 
在定义了一阶梯度求解器后，通过将model的所有输入打包成 *args 类型并输入FirstOrderGrad中，即可实现求spots的一阶梯度：

In [11]:
inputs = (expiries, strikes, spots, volatilities, discounted_rates)
Delta = grad1_solver(*inputs)
print(f"Delta={Delta}")

Delta=[ 0.02377132  0.21081336  0.59870666  0.88553166  0.98092395 -0.97622883
 -0.7891868  -0.40129346 -0.11446845 -0.01907619]


需要注意的是，若想要对定价求解器中的可选项求解希腊值(如AnalyticBlackScholesMerton类中的discounted_rates、dividend_rates)，则这些参数必须显式输入。基于BSM公式求解所有的希腊值的代码如下：

In [12]:
inputs = (expiries, strikes, spots, volatilities, discounted_rates, ms.Tensor(0.0))
print("Greek Letters Calculation for Black-Scholes-Merton Model")
# Delta
grad1 = FirstOrderGrad(model=bsm_solver, argnums=2)
print(f"Delta={grad1(*inputs)}")
# Gamma
grad2 = SecondOrderGrad(model=bsm_solver, argnums=2)
print(f"Gamma={grad2(*inputs)}")
# Theta
grad1 = FirstOrderGrad(model=bsm_solver, argnums=0)
print(f"Theta={grad1(*inputs)}")
# Vega
grad1 = FirstOrderGrad(model=bsm_solver, argnums=3)
print(f"Vega={grad1(*inputs)}")
# Rho
grad1 = FirstOrderGrad(model=bsm_solver, argnums=4)
print(f"Rho={grad1(*inputs)}")
# Psi
grad1 = FirstOrderGrad(model=bsm_solver, argnums=5)
print(f"Psi={grad1(*inputs)}")

Greek Letters Calculation for Black-Scholes-Merton Model
Delta=[ 0.02377132  0.21081336  0.59870666  0.88553166  0.98092395 -0.97622883
 -0.7891868  -0.40129346 -0.11446845 -0.01907619]
Gamma=[0.00700296 0.03209511 0.0386668  0.01758761 0.00387598 0.00700296
 0.03209511 0.0386668  0.01758761 0.00387598]
Theta=40.00596
Vega=96.00819
Rho=5.0083866
Psi=-22.93245
